In [35]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from pytorchtools import EarlyStopping
#from pytorchtools import EarlyStopping_acc
from torchsummary import summary as summary_
import datetime
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Function
from binarized_modules import  BinarizeLinear,BinarizeConv2d
from torch.autograd import Variable

In [ ]:
def quantize(a, bit=7):
    quant = a*(2**bit)
    quant = quant.round()
    quant /= 2**bit
    return quant

In [2]:
def preprocessing(person, motion):  # person, motion에 해당하는 image 불러옴
    DirectoryPath = 'C:/Users/hojung/Documents/Anaconda_python/data/class12/'
    count = 100
    image = np.zeros(shape=(count, rows, cols, 1))
    label = []
    date = '220132'
    file_name = '_stft.txt'
    cwt_data = pd.read_csv(
        DirectoryPath + date + "_" + str(person) + "_" + str(motion) + file_name)
    for i in range(0, 100):
        df = np.fromstring(cwt_data['pixels'][i], dtype=int, sep=' ')
        df = np.reshape(df, (rows, cols, 1))
        image[i] = df
        if motion == 0:
            label.append(3 * (person - 1) + 0)
        else:
            label.append(3 * (person - 1) + motion - 1)

    return image, label



# 시작과 끝 좌표는 scale한 후의 좌표를 기준으로 함
def preprocessing_resize_crop(image,start_row,end_row,start_col,end_col,row_scale,col_scale): 
    crop_image = image[:,0:image.shape[1]:row_scale,0:image.shape[2]:col_scale]
    crop_image = crop_image[:,start_row:end_row,start_col:end_col]
    return crop_image

# ratio비율로 각 data set을 합치고 순서도 섞음
def concatenate_n_div(image0, label0, image1, label1, image2, label2, image3, label3, image4, label4, image5, label5, image6, label6, image7, label7, image8, label8, image9, label9, image10, label10, image11, label11):
    count = 100
    train_ratio = 0.7
    val_ratio = 0.15
    test_ratio = 0.15  # 적용안됨

    x_train = np.concatenate(
        (image0[0:int(count*train_ratio)],
         image1[0:int(count*train_ratio)],
         image2[0:int(count*train_ratio)],
         image3[0:int(count*train_ratio)],
         image4[0:int(count*train_ratio)],
         image5[0:int(count*train_ratio)],
         image6[0:int(count*train_ratio)],
         image7[0:int(count*train_ratio)],
         image8[0:int(count*train_ratio)],
         image9[0:int(count*train_ratio)],
         image10[0:int(count*train_ratio)],
         image11[0:int(count*train_ratio)]))
    y_train = np.concatenate(
        (label0[0:int(count*train_ratio)],
         label1[0:int(count*train_ratio)],
         label2[0:int(count*train_ratio)],
         label3[0:int(count*train_ratio)],
         label4[0:int(count*train_ratio)],
         label5[0:int(count*train_ratio)],
         label6[0:int(count*train_ratio)],
         label7[0:int(count*train_ratio)],
         label8[0:int(count*train_ratio)],
         label9[0:int(count*train_ratio)],
         label10[0:int(count*train_ratio)],
         label11[0:int(count*train_ratio)]))
    x_val = np.concatenate((image0[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image1[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image2[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image3[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image4[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image5[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image6[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image7[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image8[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image9[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image10[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image11[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)]))
    y_val = np.concatenate((label0[int(count*train_ratio):int(count*train_ratio + count*val_ratio)],
                            label1[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label2[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label3[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label4[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label5[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label6[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label7[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label8[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label9[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label10[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label11[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)]))
    x_test = np.concatenate((image0[int(count*train_ratio + count*val_ratio): count],
                             image1[int(count*train_ratio +
                                        count*val_ratio): count],
                             image2[int(count*train_ratio +
                                        count*val_ratio): count],
                             image3[int(count*train_ratio +
                                        count*val_ratio): count],
                             image4[int(count*train_ratio +
                                        count*val_ratio): count],
                             image5[int(count*train_ratio +
                                        count*val_ratio): count],
                             image6[int(count*train_ratio +
                                        count*val_ratio): count],
                             image7[int(count*train_ratio + 
                                        count*val_ratio): count],
                             image8[int(count*train_ratio + 
                                        count*val_ratio): count],
                             image9[int(count*train_ratio + 
                                        count*val_ratio): count],
                             image10[int(count*train_ratio + 
                                        count*val_ratio): count],
                             image11[int(count*train_ratio + 
                                        count*val_ratio): count]))
    y_test = np.concatenate((label0[int(count*train_ratio + count*val_ratio): count],
                             label1[int(count*train_ratio +
                                        count*val_ratio): count],
                             label2[int(count*train_ratio +
                                        count*val_ratio): count],
                             label3[int(count*train_ratio +
                                        count*val_ratio): count],
                             label4[int(count*train_ratio +
                                        count*val_ratio): count],
                             label5[int(count*train_ratio +
                                        count*val_ratio): count],
                             label6[int(count*train_ratio +
                                        count*val_ratio): count],
                             label7[int(count*train_ratio + 
                                        count*val_ratio): count],
                             label8[int(count*train_ratio + 
                                        count*val_ratio): count],
                             label9[int(count*train_ratio + 
                                        count*val_ratio): count],
                             label10[int(count*train_ratio + 
                                        count*val_ratio): count],
                             label11[int(count*train_ratio + 
                                        count*val_ratio): count]))


    s = np.arange(x_train.shape[0])
    np.random.shuffle(s)
    x_train = x_train[s]
    y_train = y_train[s]

    s = np.arange(x_val.shape[0])
    np.random.shuffle(s)
    x_val = x_val[s]
    y_val = y_val[s]

    s = np.arange(x_test.shape[0])
    np.random.shuffle(s)
    x_test = x_test[s]
    y_test = y_test[s]

    return x_train, y_train, x_val, y_val, x_test, y_test

class TensorData(Dataset):
    
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.x_data = self.x_data.permute(0,3,1,2) # 이미지 개수, 채널 수, 이미지 너비, 높이
        self.y_data = torch.LongTensor(y_data)
        self.len = self.y_data.shape[0]
        
    def __getitem__(self, index):
        
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [3]:
class Net(nn.Module):

    def __init__(self, num_classes=4):
        super(Net,self).__init__()
        self.infl_ratio=3;
        self.channel = 16;
        self.features = nn.Sequential(
            # BinarizeConv2d(입력 채널수, 출력 채널수, 커널 사이즈, 스트라이드, 패딩)
            BinarizeConv2d(1, self.channel*self.infl_ratio,kernel_size=3,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=3, stride=1),
            # nn.BatchNorm2d(입출력 채널수)
            nn.BatchNorm2d(self.channel*self.infl_ratio),
            nn.Hardtanh(),

            BinarizeConv2d(self.channel*self.infl_ratio, self.channel*self.infl_ratio,kernel_size=3,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.BatchNorm2d(self.channel*self.infl_ratio),
            nn.Hardtanh(),

            BinarizeConv2d(self.channel*self.infl_ratio, self.channel*self.infl_ratio,kernel_size=3,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.BatchNorm2d(self.channel*self.infl_ratio),
            nn.Hardtanh(),

            BinarizeConv2d(self.channel*self.infl_ratio, self.channel,kernel_size=3,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.BatchNorm2d(self.channel),
            nn.Hardtanh()

         )
            
        self.classifier = nn.Sequential(
            # BinarizeLinear(입력 dense길이, 출력 dense길이)
            BinarizeLinear(self.channel*20*12,32),
            BinarizeLinear(32,num_classes)
        )
        self.softmax = nn.Softmax()
                
                
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1,self.channel*20*12)
        x = self.classifier(x)
        return self.softmax(x)

In [4]:
class Net2(nn.Module):

    def __init__(self, num_classes=4):
        super(Net2,self).__init__()
        self.infl_ratio=3;
        
            # BinarizeConv2d(입력 채널수, 출력 채널수, 커널 사이즈, 스트라이드, 패딩)
        self.conv_L1 = BinarizeConv2d(1, 16*self.infl_ratio,kernel_size=3,stride=1,padding='same')
        self.max_L1 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.bn_L1 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L1 = nn.Hardtanh()
        self.conv_R1 = BinarizeConv2d(1, 16*self.infl_ratio,kernel_size=5,stride=1,padding='same')
        self.max_R1 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.bn_R1 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_R1 = nn.Hardtanh()

        self.max1 = nn.MaxPool2d(kernel_size=2, stride=1)

        self.conv_L2 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=3,stride=1,padding='same')
        self.max_L2 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.bn_L2 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L2 = nn.Hardtanh()
        self.conv_R2 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=5,stride=1,padding='same')
        self.max_R2 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.bn_R2 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_R2 = nn.Hardtanh()
        
        
        
        self.fc1 = BinarizeLinear(48*33*25,64)
        self.human = BinarizeLinear(64,4)
         
            
        
        self.softmax2 = nn.Softmax()
                
                
    def forward(self, x):
        x1 = self.conv_L1(x)
        x1 = self.max_L1(x1)
        x1 = self.bn_L1(x1)
        x1 = self.htan_L1(x1)

        x2 = self.conv_R1(x)
        x2 = self.max_R1(x2)
        x2 = self.bn_R1(x2)
        x2 = self.htan_R1(x2)

        x3 = torch.add(x1,x2)
        x3 = self.max1(x3)

        x4 = self.conv_L2(x3)
        x4 = self.max_L2(x4)
        x4 = self.bn_L2(x4)
        x4 = self.htan_L2(x4)

        x5 = self.conv_R2(x3)
        x5 = self.max_R2(x5)
        x5 = self.bn_R2(x5)
        x5 = self.htan_R2(x5)

        x6 = torch.add(x4,x5)

        x6 = x6.view(-1,48*33*25)
        x6 = self.fc1(x6)
        human = self.softmax2(self.human(x6))
        


        
        return human

In [26]:
class Net3(nn.Module):

    def __init__(self, num_classes=4):
        super(Net3,self).__init__()
        self.infl_ratio=3;
        
            # BinarizeConv2d(입력 채널수, 출력 채널수, 커널 사이즈, 스트라이드, 패딩)
        self.conv_L1 = BinarizeConv2d(1, 16*self.infl_ratio,kernel_size=3,stride=1,padding=0)
        self.max_L1 = nn.MaxPool2d(kernel_size=3, stride=1)
        self.bn_L1 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L1 = nn.Hardtanh()
        # self.conv_R1 = BinarizeConv2d(1, 16*self.infl_ratio,kernel_size=5,stride=1,padding='same')
        # self.max_R1 = nn.MaxPool2d(kernel_size=2, stride=1)
        # self.bn_R1 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        # self.htan_R1 = nn.Hardtanh()

        # self.max1 = nn.MaxPool2d(kernel_size=2, stride=1)

        self.conv_L2 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=3,stride=1,padding=0)
        self.max_L2 = nn.MaxPool2d(kernel_size=3, stride=1)
        self.bn_L2 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L2 = nn.Hardtanh()
        # self.conv_R2 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=5,stride=1,padding='same')
        # self.max_R2 = nn.MaxPool2d(kernel_size=2, stride=1)
        # self.bn_R2 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        # self.htan_R2 = nn.Hardtanh()
        
        
        self.fc1 = BinarizeLinear(16*self.infl_ratio*28*20,128)
        self.human = BinarizeLinear(128,12)
         
            
        
        self.softmax2 = nn.Softmax()
                
                
    def forward(self, x):
        x1 = self.conv_L1(x)
        x1 = self.max_L1(x1)
        x1 = self.bn_L1(x1)
        x1 = self.htan_L1(x1)
        
        # x2 = self.conv_R1(x)
        # x2 = self.max_R1(x2)
        # x2 = self.bn_R1(x2)
        # x2 = self.htan_R1(x2)

        # x3 = torch.add(x1,x2)
        # x3 = self.max1(x3)

        x1 = self.conv_L2(x1)
        x1 = self.max_L2(x1)
        x1 = self.bn_L2(x1)
        x1 = self.htan_L2(x1)
        
        # x5 = self.conv_R2(x3)
        # x5 = self.max_R2(x5)
        # x5 = self.bn_R2(x5)
        # x5 = self.htan_R2(x5)

        

        x1 = x1.view(-1,16*self.infl_ratio*28*20)
        x1 = self.fc1(x1)
        human = self.softmax2(self.human(x1))
        


        
        return human

In [27]:
class Net2(nn.Module):

    def __init__(self, num_classes=4):
        super(Net2,self).__init__()
        self.infl_ratio=3;
        
            # BinarizeConv2d(입력 채널수, 출력 채널수, 커널 사이즈, 스트라이드, 패딩)
        self.conv_L1 = BinarizeConv2d(1, 16*self.infl_ratio,kernel_size=3,stride=1,padding='same')
        self.max_L1 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.bn_L1 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L1 = nn.Hardtanh()
        self.conv_R1 = BinarizeConv2d(1, 16*self.infl_ratio,kernel_size=5,stride=1,padding='same')
        self.max_R1 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.bn_R1 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_R1 = nn.Hardtanh()

        self.max1 = nn.MaxPool2d(kernel_size=2, stride=1)

        self.conv_L2 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=3,stride=1,padding='same')
        self.max_L2 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.bn_L2 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L2 = nn.Hardtanh()
        self.conv_R2 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=5,stride=1,padding='same')
        self.max_R2 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.bn_R2 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_R2 = nn.Hardtanh()
        
        self.max2 = nn.MaxPool2d(kernel_size=2, stride=1)

        self.conv_L3 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=3,stride=1,padding='same')
        self.max_L3 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.bn_L3 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L3 = nn.Hardtanh()
        self.conv_R3 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=5,stride=1,padding='same')
        self.max_R3 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.bn_R3 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_R3 = nn.Hardtanh()
        
        self.max3 = nn.MaxPool2d(kernel_size=2, stride=1)
        
        self.fc1 = BinarizeLinear(48*30*22,128)
        self.human = BinarizeLinear(128,4)
        #self.motion = BinarizeLinear(128,3)
            
        self.softmax1 = nn.Softmax()
        self.softmax2 = nn.Softmax()
                
                
    def forward(self, x):
        x1 = self.conv_L1(x)
        x1 = self.max_L1(x1)
        x1 = self.bn_L1(x1)
        x1 = self.htan_L1(x1)

        x2 = self.conv_R1(x)
        x2 = self.max_R1(x2)
        x2 = self.bn_R1(x2)
        x2 = self.htan_R1(x2)

        x3 = torch.add(x1,x2)
        x3 = self.max1(x3)

        x4 = self.conv_L2(x3)
        x4 = self.max_L2(x4)
        x4 = self.bn_L2(x4)
        x4 = self.htan_L2(x4)

        x5 = self.conv_R2(x3)
        x5 = self.max_R2(x5)
        x5 = self.bn_R2(x5)
        x5 = self.htan_R2(x5)

        x6 = torch.add(x4,x5)
        x6 = self.max2(x6)
        
        x7 = self.conv_L3(x6)
        x7 = self.max_L3(x7)
        x7 = self.bn_L3(x7)
        x7 = self.htan_L3(x7)

        x8 = self.conv_R3(x6)
        x8 = self.max_R3(x8)
        x8 = self.bn_R3(x8)
        x8 = self.htan_R3(x8)

        x9 = torch.add(x7,x8)
        x9 = self.max3(x9)
        
        x9 = x9.view(-1,48*30*22)
        x9 = self.fc1(x9)
        human = self.softmax2(self.human(x9))
        #motion = self.softmax1(self.motion(x9))
        


        
        return human

In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [29]:
net = Net3().to(device)
summary_(net,(1,36,28),batch_size=32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
    BinarizeConv2d-1           [32, 48, 34, 26]             480
         MaxPool2d-2           [32, 48, 32, 24]               0
       BatchNorm2d-3           [32, 48, 32, 24]              96
          Hardtanh-4           [32, 48, 32, 24]               0
    BinarizeConv2d-5           [32, 48, 30, 22]          20,784
         MaxPool2d-6           [32, 48, 28, 20]               0
       BatchNorm2d-7           [32, 48, 28, 20]              96
          Hardtanh-8           [32, 48, 28, 20]               0
    BinarizeLinear-9                  [32, 128]       3,440,768
   BinarizeLinear-10                   [32, 12]           1,548
          Softmax-11                   [32, 12]               0
Total params: 3,463,772
Trainable params: 3,463,772
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.12
Forw

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_10036/4075584493.py:65: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  human = self.softmax2(self.human(x1))


In [9]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device), 

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.data.copy_(p.org)
        optimizer.step()
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.org.copy_(p.data.clamp_(-1,1))
                
        if batch % 40 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred,y)
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy_human: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return 100*correct

In [10]:
# SCALE:16 /1초 추출 + 0.2초 씩 이동 / row 30
#=========================================================
#================파라미터 설정==========================
#============================================================
# STFT = (128, 29) //  CWT = (81,1920)
# 좌표는 스케일 이후 범위
start_row = 46
end_row = 82
scale_row = 1
rows = 128

start_col = 0
end_col = 28
scale_col = 1
cols = 29

aug = 1  # 1이면 augmentation 0이면 X

gen = ImageDataGenerator(
                         width_shift_range=0.2
                            )


image1, label1 = preprocessing(1, 0)
image2, label2 = preprocessing(1, 2)
image3, label3 = preprocessing(1, 3)
image4, label4 = preprocessing(2, 0)
image5, label5 = preprocessing(2, 2)
image6, label6 = preprocessing(2, 3)
image7, label7 = preprocessing(3, 0)
image8, label8 = preprocessing(3, 2)
image9, label9 = preprocessing(3, 3)
image10, label10 = preprocessing(4, 0)
image11, label11 = preprocessing(4, 2)
image12, label12 = preprocessing(4, 3)

# 정규화 추가
total_img = np.concatenate((image1, image2, image3, image4, image5, image6, image7,
 image8, image9, image10, image11, image12))
total_std = total_img.std()
total_mean = total_img.mean()



In [32]:
#torch.cuda.empty_cache()
epochs = 100  # 학습을 원하는 만큼 수정요
try_num = 3
human_result_acc = np.zeros(try_num*2*epochs).reshape(try_num,2,epochs)
motion_result_acc = np.zeros(try_num*2*epochs).reshape(try_num,2,epochs)
average_result_acc = np.zeros(try_num*epochs).reshape(try_num,epochs)
average_acc = 0
quanta = 1
start = datetime.datetime.now()
for a in range(try_num):

    model = Net3()
    model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    s = np.arange(image1.shape[0])
    np.random.shuffle(s)
    image1_shuff = image1[s]

    s = np.arange(image2.shape[0])
    np.random.shuffle(s)
    image2_shuff = image2[s]

    s = np.arange(image3.shape[0])
    np.random.shuffle(s)
    image3_shuff = image3[s]

    s = np.arange(image4.shape[0])
    np.random.shuffle(s)
    image4_shuff = image4[s]

    s = np.arange(image5.shape[0])
    np.random.shuffle(s)
    image5_shuff = image5[s]

    s = np.arange(image6.shape[0])
    np.random.shuffle(s)
    image6_shuff = image6[s]

    s = np.arange(image7.shape[0])
    np.random.shuffle(s)
    image7_shuff = image7[s]

    s = np.arange(image8.shape[0])
    np.random.shuffle(s)
    image8_shuff = image8[s]

    s = np.arange(image9.shape[0])
    np.random.shuffle(s)
    image9_shuff = image9[s]

    s = np.arange(image10.shape[0])
    np.random.shuffle(s)
    image10_shuff = image10[s]

    s = np.arange(image11.shape[0])
    np.random.shuffle(s)
    image11_shuff = image11[s]

    s = np.arange(image12.shape[0])
    np.random.shuffle(s)
    image12_shuff = image12[s]

    # 크기에 맞게 자름
    image1_crop = preprocessing_resize_crop(
        image1_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image2_crop = preprocessing_resize_crop(
        image2_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image3_crop = preprocessing_resize_crop(
        image3_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image4_crop = preprocessing_resize_crop(
        image4_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image5_crop = preprocessing_resize_crop(
        image5_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image6_crop = preprocessing_resize_crop(
        image6_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image7_crop = preprocessing_resize_crop(
        image7_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image8_crop = preprocessing_resize_crop(
        image8_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image9_crop = preprocessing_resize_crop(
        image9_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image10_crop = preprocessing_resize_crop(
        image10_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image11_crop = preprocessing_resize_crop(
        image11_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image12_crop = preprocessing_resize_crop(
        image12_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)

    # 자른 image를 각 data set으로 나눠서 합침
    x_train, y_train, x_val, y_val, x_test, y_test = concatenate_n_div(
        image1_crop, label1, image2_crop, label2, image3_crop, label3, image4_crop, label4, image5_crop, 
        label5, image6_crop, label6, image7_crop, label7, image8_crop, label8, image9_crop, label9, image10_crop, 
        label10, image11_crop, label11, image12_crop, label12)


    x_train = (x_train - total_mean) / total_std
    x_val = (x_val - total_mean) / total_std
    x_test = (x_test - total_mean) / total_std

    # 보강할 학습데이터 이미지 생성
    if aug == 1:
        augment_ratio = 9   # 전체 데이터의 150%
        augment_size = int(augment_ratio * x_train.shape[0])

        # 전체 x_train 개수의 150% 비율만큼
        randidx = np.random.randint(x_train.shape[0], size=augment_size)

        # 임의로 선택된 데이터는 원본데이터를 참조하기 때문에
        # 원본데이터에 영향을 줄수 있음. 그래서 copy() 함수를 통해 안전하게 복사본 만듬
        x_augmented = x_train[randidx].copy()  
        y_augmented = y_train[randidx].copy()

        #  이미지 보강 실행
        x_augmented, y_augmented = gen.flow(x_augmented, y_augmented, 
                                            batch_size=augment_size,
                                            shuffle=False).next()

        x_train = np.concatenate((x_train,x_augmented))
        y_train = np.concatenate((y_train,y_augmented))
        s = np.arange(x_train.shape[0])
        np.random.shuffle(s)
        x_train = x_train[s]
        y_train = y_train[s]

    if quanta == 1:
        x_train = quantize(x_train)
        x_val = quantize(x_val)
        x_test = quantize(x_test)

    train_data = TensorData(x_train,y_train)
    test_data = TensorData(x_test,y_test)
    valid_data = TensorData(x_val,y_val)
    train_loader = DataLoader(train_data, batch_size = 32, shuffle =True)
    test_loader = DataLoader(test_data, batch_size = 32, shuffle =True)
    valid_loader = DataLoader(valid_data, batch_size = 32, shuffle =True)

    for t in range(epochs):
        print(f"total try_num {a+1}, Epoch {t+1}\n-------------------------------")
        train(train_loader, model, criterion, optimizer)

        human_result_acc[a][0][t] = test(valid_loader, model, criterion)
        human_result_acc[a][1][t] = test(test_loader, model, criterion)
        average_result_acc[a][t] = (human_result_acc[a][0][t] + human_result_acc[a][1][t]) / 2
print("Done!")
end = datetime.datetime.now()
print('동작시간 :', end - start)
for b in range(try_num):
    print(f"try : {b+1} // epoch : {np.argmax(average_result_acc[b])+1} // Max_average_accuracy = {max(average_result_acc[b]):0.1f},\
 Max_accuray_valid_human = {human_result_acc[b][0][np.argmax(average_result_acc[b])]:0.1f},\
 Max_accuray_test_human = {human_result_acc[b][1][np.argmax(average_result_acc[b])]:0.1f}\n ")
    average_acc += max(average_result_acc[b])
average_acc /= try_num
print(f"total average_acc = {average_acc:0.1f}")
    

total try_num 1, Epoch 1
-------------------------------
loss: 2.550032  [    0/ 8400]


C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_10036/4075584493.py:65: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  human = self.softmax2(self.human(x1))


loss: 2.461491  [ 1280/ 8400]
loss: 2.334968  [ 2560/ 8400]
loss: 2.274488  [ 3840/ 8400]
loss: 2.274578  [ 5120/ 8400]
loss: 2.454208  [ 6400/ 8400]
loss: 2.337479  [ 7680/ 8400]
Test Error: 
 Accuracy_human: 30.6%, Avg loss: 0.077284 

Test Error: 
 Accuracy_human: 31.1%, Avg loss: 0.076644 

total try_num 1, Epoch 2
-------------------------------
loss: 2.399950  [    0/ 8400]
loss: 2.368650  [ 1280/ 8400]
loss: 2.306219  [ 2560/ 8400]
loss: 2.368719  [ 3840/ 8400]
loss: 2.243729  [ 5120/ 8400]
loss: 2.243660  [ 6400/ 8400]
loss: 2.306091  [ 7680/ 8400]
Test Error: 
 Accuracy_human: 33.3%, Avg loss: 0.076036 

Test Error: 
 Accuracy_human: 32.8%, Avg loss: 0.076509 

total try_num 1, Epoch 3
-------------------------------
loss: 2.274490  [    0/ 8400]
loss: 2.181722  [ 1280/ 8400]
loss: 2.243649  [ 2560/ 8400]
loss: 2.184134  [ 3840/ 8400]
loss: 2.180809  [ 5120/ 8400]
loss: 2.180819  [ 6400/ 8400]
loss: 2.117755  [ 7680/ 8400]
Test Error: 
 Accuracy_human: 32.8%, Avg loss: 0.07617

In [33]:
print('동작시간 :', end - start)
for b in range(try_num):
    print(f"try : {b+1} // epoch : {np.argmax(average_result_acc[b])+1} // Max_average_accuracy = {max(average_result_acc[b]):0.1f},\
 Max_accuray_valid_human = {human_result_acc[b][0][np.argmax(average_result_acc[b])]:0.1f},\
 Max_accuray_test_human = {human_result_acc[b][1][np.argmax(average_result_acc[b])]:0.1f}\n ")

print(f"total average_acc = {average_acc:0.1f}")

동작시간 : 0:07:50.816556
try : 1 // epoch : 98 // Max_average_accuracy = 78.6, Max_accuray_valid_human = 78.9, Max_accuray_test_human = 78.3
 
try : 2 // epoch : 96 // Max_average_accuracy = 61.9, Max_accuray_valid_human = 62.2, Max_accuray_test_human = 61.7
 
try : 3 // epoch : 92 // Max_average_accuracy = 61.4, Max_accuray_valid_human = 61.7, Max_accuray_test_human = 61.1
 
total average_acc = 67.3


In [13]:
dict(model.named_parameters())

{'conv_L1.weight': Parameter containing:
 tensor([[[[-1., -1.,  1.],
           [ 1., -1.,  1.],
           [ 1., -1.,  1.]]],
 
 
         [[[-1., -1.,  1.],
           [-1., -1., -1.],
           [ 1., -1., -1.]]],
 
 
         [[[ 1., -1., -1.],
           [-1.,  1.,  1.],
           [-1.,  1.,  1.]]],
 
 
         [[[-1.,  1.,  1.],
           [ 1.,  1.,  1.],
           [ 1., -1., -1.]]],
 
 
         [[[-1.,  1., -1.],
           [-1.,  1.,  1.],
           [ 1., -1., -1.]]],
 
 
         [[[ 1., -1.,  1.],
           [ 1.,  1., -1.],
           [-1.,  1.,  1.]]],
 
 
         [[[ 1., -1.,  1.],
           [ 1., -1., -1.],
           [-1.,  1.,  1.]]],
 
 
         [[[-1., -1.,  1.],
           [ 1.,  1., -1.],
           [ 1., -1.,  1.]]],
 
 
         [[[-1., -1.,  1.],
           [ 1., -1., -1.],
           [ 1., -1., -1.]]],
 
 
         [[[-1., -1.,  1.],
           [ 1.,  1.,  1.],
           [ 1.,  1., -1.]]],
 
 
         [[[-1., -1.,  1.],
           [ 1., -1., -1.],
   